## Data wrangling


In [1]:
#Importing all the libraries I believe I will be using
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from library.sb_utils import save_file
from urllib.request import urlretrieve
from pandas_profiling import ProfileReport

The file I am extracting was taken from [A clinical decision support system learned from data to personalize treatment recommendations towards preventing breast cancer metastasis](https://datadryad.org/stash/dataset/doi%253A10.5061%252Fdryad.64964m0), which contains the following information:

|Variable  ||Description                                ||Values           |
|----------||-------------||----|
|metastasis||Whether patient metastasized within 5 years||yes, no          |
|age	   ||age at diagnosis of the disease            ||0-49, 50-64, > 64|
|menopause ||inferred menopausal status		||pre, post|
|size	   ||size of tumor in mm			||0-38, 38-50.5, > 50.5|
|node_positive	||number of positive lymph nodes		||0, 1-3,  > 3|
|node_removed	||number of lymph nodes removed		||0-2, 3-5 > 5|
|node_status	||patient had any positive lymph nodes	||neg,pos|
|grade		||grade of disease			||1, 2, 3|
|invasive	||whether tumor is invasive		||yes,no|
|stage		||composite of size and # positive nodes	||0,1,2,3|
|histology	||tumor histology				||lobular, duct|
|ER		    ||estrogen receptor expression		||neg, pos|
|PR		    ||progesterone receptor expression	||neg, pos|
|HER2		||HER2 expression	||neg, pos|
|TNEG		||patient ER, PR, and HER2 negative	||yes, no|
|P53		||whether P53 is mutated	||neg, pos|
|surgical_margins||Whether there is a residual tumor after surgery	||res. tumor, no res. tumor,no primary site surgery|
|surgery	||type of surgery				||conservation, mastectomy|
|chemo	||whether patient had chemotherapy	||yes, no|
|breast_chest_radi	||whether patient had breast or chest radiation	||yes, no|
|nodal_radi	||whether patient had lymph node  radiation 	||yes, no|
|antihormone	||whether patient had hormone therapy	||yes, no|
|HER2_Inhib	||whether patient had a HER2 inhibitor	||yes, no|
|neo		||Whether patient had neoadjuvant therapy	||yes, no|

In [2]:
#Let's downlaod the file directly from the official site
url = 'https://datadryad.org/stash/downloads/file_stream/83536'
file_name = 'SourceFiles/'+'LSDS-5YDM.txt'
urlretrieve(url,file_name)

('SourceFiles/LSDS-5YDM.txt', <http.client.HTTPMessage at 0x17850838a20>)

In [3]:
data = pd.read_csv(file_name,sep='\t')
data.head()

,metastasis,age,menopause,TNEG,ER,PR,P53,HER2,stage,node_status,...,grade,invasive,surgical_margins,surgery,breast_chest_radi,nodal_radi,chemo,antihormone,HER2_Inhib,neo
0,NO,three,POST,NO,POSITIVE,POSITIVE,NEGATIVE,NEGATIVE,one,NEGATIVE,...,two,YES,NO RESIDUAL TUMOR,CONSERVATION,YES,NO,NO,YES,NO,NO
1,NO,two,POST,NO,POSITIVE,POSITIVE,NEGATIVE,NEGATIVE,three,POSITIVE,...,one,YES,NO RESIDUAL TUMOR,CONSERVATION,NO,NO,YES,NO,NO,NO
2,NO,one,PRE,NO,POSITIVE,POSITIVE,NEGATIVE,NEGATIVE,one,POSITIVE,...,one,YES,NO RESIDUAL TUMOR,CONSERVATION,YES,NO,NO,NO,NO,NO
3,NO,three,POST,NO,POSITIVE,POSITIVE,NEGATIVE,NEGATIVE,one,NEGATIVE,...,one,YES,NO RESIDUAL TUMOR,CONSERVATION,NO,NO,NO,YES,NO,NO
4,NO,three,POST,NO,POSITIVE,POSITIVE,NEGATIVE,NEGATIVE,one,NEGATIVE,...,two,YES,NO RESIDUAL TUMOR,CONSERVATION,YES,NO,NO,YES,NO,NO


In [4]:
#Let's analyze if we have values for all rows and columns
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6726 entries, 0 to 6725
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   metastasis         6726 non-null   object
 1   age                6726 non-null   object
 2   menopause          6726 non-null   object
 3   TNEG               6726 non-null   object
 4   ER                 6726 non-null   object
 5   PR                 6726 non-null   object
 6   P53                6726 non-null   object
 7   HER2               6726 non-null   object
 8   stage              6726 non-null   object
 9   node_status        6726 non-null   object
 10  node_positive      6726 non-null   object
 11  node_removed       6726 non-null   object
 12  histology          6726 non-null   object
 13  size               6726 non-null   object
 14  grade              6726 non-null   object
 15  invasive           6726 non-null   object
 16  surgical_margins   6726 non-null   object


I need some work here, all the columns are categorical, but right now, the dtype is Object. Also, we need to make them numerical by spliting the columns as needed.

In [5]:
#Defining a transformation function
#To change YES to 1 and NO to 0
def YES_NO(col):
    if col == 'YES':
        return 1
    elif col == 'NO':
        return 0
    else:
        return col

In [6]:
#Let's apply the YES_NO to all YES/NO columns
#These are ['metastasis', 'TNEG', 'invasive', 'breast_chest_radi', 
#           'nodal_radi', 'chemo', 'antihormone', 'HER2_Inhib', 'neo']
df = pd.DataFrame([], columns=['METASTASIS', 'TNEG', 'INVASIVE','BREAST_CHEST_RADI','NODAL_RADI','CHEMO','ANTIHORMONE','HER2_INHIB','NEO','ER', 'PR', 'P53', 'HER2', 'NODE_STATUS'])
df['METASTASIS'] = data['metastasis'].apply(YES_NO)
df['TNEG'] = data['TNEG'].apply(YES_NO)
df['INVASIVE'] = data['invasive'].apply(YES_NO)
df['BREAST_CHEST_RADI'] = data['breast_chest_radi'].apply(YES_NO)
df['NODAL_RADI'] = data['nodal_radi'].apply(YES_NO)
df['CHEMO'] = data['chemo'].apply(YES_NO)
df['ANTIHORMONE'] = data['antihormone'].apply(YES_NO)
df['HER2_INHIB'] = data['HER2_Inhib'].apply(YES_NO)
df['NEO'] = data['neo'].apply(YES_NO)

In [7]:
#We will do a similar process for the columns with values POSITIVE/NEGATIVE
def POS_NEG(col):
    if col == 'POSITIVE':
        return 1
    elif col == 'NEGATIVE':
        return -1
    else:
        return col

In [8]:
#Applying the POS_NEG to the columns
# ['ER', 'PR', 'P53', 'HER2', 'node_status']
df['ER'] = data['ER'].apply(POS_NEG)
df['PR'] = data['PR'].apply(POS_NEG)
df['P53'] = data['P53'].apply(POS_NEG)
df['HER2'] = data['HER2'].apply(POS_NEG)
df['NODE_STATUS'] = data['node_status'].apply(POS_NEG)

In [9]:
#We will do a similar process for the columns with True/False
def T_F(col):
    if col == True:
        return 1
    elif col == False:
        return 0
    else:
        return col

In [10]:
def SEQUENCE(col):
    if col == 'one':
        return 1
    elif col == 'two':
        return 2
    elif col == 'three':
        return 3
    else:
        return col

In [27]:
#For the remaining columns, we will create new columns according to the value they hold
#age
age_0_to_49 = data['age'] == 'one'
age_50_to_64 = data['age'] == 'two'
age_65_or_more = data['age'] == 'three'
df['AGE_0_TO_49'] = age_0_to_49.apply(T_F)
df['AGE_50_TO_64'] = age_50_to_64.apply(T_F)
df['AGE_65_OR_MORE'] = age_65_or_more.apply(T_F)

#menopause
pre_menopause = data['menopause'] == 'PRE'
post_menopause = data['menopause'] == 'POST'
df['PRE_MENOPAUSE'] = pre_menopause.apply(T_F)
df['POST_MENOPAUSE'] = post_menopause.apply(T_F)

#node_positive
no_nodes = data['node_positive'] == 'zero'
nodes_1_to_3 = data['node_positive'] == 'one'
nodes_4_or_more = data['node_positive'] == 'two'
df['NO_NODES'] = no_nodes.apply(T_F)
df['NODES_1_TO_3'] = nodes_1_to_3.apply(T_F)
df['NODES_4_OR_MORE'] = nodes_4_or_more.apply(T_F)

#node_removed
nodes_removed_0_to_2 = data['node_removed'] == 'one'
nodes_removed_3_to_5 = data['node_removed'] == 'two'
nodes_removed_6_or_more = data['node_removed'] == 'three'
df['NODES_REMOVED_0_TO_2'] = nodes_removed_0_to_2.apply(T_F)
df['NODES_REMOVED_3_TO_5'] = nodes_removed_3_to_5.apply(T_F)
df['NODES_REMOVED_6_OR_MORE'] = nodes_removed_6_or_more.apply(T_F)

#histology
tumor_duct = data['histology'] == 'DUCT'
tumor_lobular = data['histology'] == 'LOBULAR'
df['TUMOR_DUCT'] = tumor_duct.apply(T_F)
df['TUMOR_LOBULAR'] = tumor_lobular.apply(T_F)

#size, 0-38, 38-50.5, > 50.5
size_0_to_38 = data['size'] == 'one'
size_38_to_50 = data['size'] == 'two'
size_50_or_more = data['size'] == 'three'
df['SIZE_0_TO_38'] = size_0_to_38.apply(T_F)
df['SIZE_38_TO_50.5'] = size_38_to_50.apply(T_F)
df['SIZE_50.5_OR_MORE'] = size_50_or_more.apply(T_F)

#grade
df['GRADE'] = data['grade'].apply(SEQUENCE)

#surgical_margins, NO RESIDUAL TUMOR, RESIDUAL TUMOR, NO PRIMARY SITE
surgery_no_residual_margin = data['surgical_margins'] == 'NO RESIDUAL TUMOR'
residual_margin_after_surgery = data['surgical_margins'] == 'RESIDUAL TUMOR'
autopsy = data['surgical_margins'] == 'NO PRIMARY SITE'
df['SURGERY_NO_RESIDUAL'] = surgery_no_residual_margin.apply(T_F)
df['SURGERY_RESIDUAL'] = residual_margin_after_surgery.apply(T_F)
df['AUTOPSY'] = autopsy.apply(T_F)

#surgery, CONSERVATION, MASTECTOMY, NO SURGERY
conservation = data['surgery'] == 'CONSERVATION'
mastectomy = data['surgery'] == 'MASTECTOMY'
no_surgery = data['surgery'] == 'NO SURGERY'
df['CONSERVATION_SURGERY'] = conservation.apply(T_F)
df['MASTECTOMY_SURGERY'] = mastectomy.apply(T_F)
df['NO_SURGERY'] = no_surgery.apply(T_F)
df.columns = df.columns.str.lower()
df.head()

,metastasis,tneg,invasive,breast_chest_radi,nodal_radi,chemo,antihormone,her2_inhib,neo,er,...,size_0_to_38,size_38_to_50.5,size_50.5_or_more,grade,surgery_no_residual,surgery_residual,autopsy,conservation_surgery,mastectomy_surgery,no_surgery
0,0,0,1,1,0,0,1,0,0,1,...,1,0,0,2,1,0,0,1,0,0
1,0,0,1,0,0,1,0,0,0,1,...,0,0,1,1,1,0,0,1,0,0
2,0,0,1,1,0,0,0,0,0,1,...,1,0,0,1,1,0,0,1,0,0
3,0,0,1,0,0,0,1,0,0,1,...,1,0,0,1,1,0,0,1,0,0
4,0,0,1,1,0,0,1,0,0,1,...,1,0,0,2,1,0,0,1,0,0


The variables "stage" will continue with it's original value since it does not require any tranformation.

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6726 entries, 0 to 6725
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   metastasis               6726 non-null   int64
 1   tneg                     6726 non-null   int64
 2   invasive                 6726 non-null   int64
 3   breast_chest_radi        6726 non-null   int64
 4   nodal_radi               6726 non-null   int64
 5   chemo                    6726 non-null   int64
 6   antihormone              6726 non-null   int64
 7   her2_inhib               6726 non-null   int64
 8   neo                      6726 non-null   int64
 9   er                       6726 non-null   int64
 10  pr                       6726 non-null   int64
 11  p53                      6726 non-null   int64
 12  her2                     6726 non-null   int64
 13  node_status              6726 non-null   int64
 14  age_0_to_49              6726 non-null   int64
 15  age_

In [29]:
#Let's now analyze if we have consistency on the values across the rows
for col in df.columns:
    print(df[col].value_counts())
    print('------------------')

0    6193
1     533
Name: metastasis, dtype: int64
------------------
0    6011
1     715
Name: tneg, dtype: int64
------------------
1    5722
0    1004
Name: invasive, dtype: int64
------------------
1    3400
0    3326
Name: breast_chest_radi, dtype: int64
------------------
0    5920
1     806
Name: nodal_radi, dtype: int64
------------------
0    4858
1    1868
Name: chemo, dtype: int64
------------------
0    4464
1    2262
Name: antihormone, dtype: int64
------------------
0    6457
1     269
Name: her2_inhib, dtype: int64
------------------
0    6411
1     315
Name: neo, dtype: int64
------------------
 1    5462
-1    1264
Name: er, dtype: int64
------------------
 1    4813
-1    1913
Name: pr, dtype: int64
------------------
-1    5230
 1    1496
Name: p53, dtype: int64
------------------
-1    5866
 1     860
Name: her2, dtype: int64
------------------
-1    4771
 1    1955
Name: node_status, dtype: int64
------------------
0    4635
1    2091
Name: age_0_to_49, dtype: int6

As we can see, the data is clean from nulls or missing values

In [30]:
#Let's get some basic statistics on each field
for col in df.columns:
    print(df[col].describe())
    print('------------------')

count    6726.000000
mean        0.079245
std         0.270140
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: metastasis, dtype: float64
------------------
count    6726.000000
mean        0.106304
std         0.308249
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: tneg, dtype: float64
------------------
count    6726.000000
mean        0.850729
std         0.356382
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: invasive, dtype: float64
------------------
count    6726.000000
mean        0.505501
std         0.500007
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: breast_chest_radi, dtype: float64
------------------
count    6726.000000
mean        0.119833
std         0.324791
min         0.000000
25%         0.000000
50%         0.00

In [31]:
profile = ProfileReport(df, title="Pandas Profiling Report",explorative=True)

In [32]:
profile.set_variable("samples", None)
profile.set_variable("duplicates", None)

In [33]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/50 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]